In [ ]:
import requests
import pandas as pd
import json
import ast

class readURL:
    data: pd.DataFrame

    def __init__(self, url: str):
        self.url = url
        
    def read(self):
        payload = ""
        headers = {}
        response = requests.request("GET", self.url, headers=headers, data=payload)
        return response
    
    def text(self):
        read = self.read().text
        return read

    def only_dict(self, d):
        return ast.literal_eval(d)

    def list_of_dicts(self, ld):
        return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

    def expand(self, withId:bool = True) -> pd.DataFrame:
        collumns = self.data.columns
        for col in collumns:
            if self.data[col].dtype == object and self.data[col].name != '_id' :
                #if withId:
                newColumn = self.data[col].tolist()
                print(newColumn)
                #newColumn = pd.json_normalize(newColumn).add_prefix(str(self.data[col].name) + '.')
                #else:
                    #newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).drop(columns=['_id']).add_prefix(str(data[x].name) + '.')
                #data = data.drop(columns=data[x].name)
                #data = data.join([newColumn])
        return self.data

    def dataFrame(self, withId:bool = True, columnExpand:bool = True) -> pd.DataFrame:
        self.data = pd.read_json(self.text())
        if not withId:
            self.data = self.data.drop(columns=['_id'])
        if columnExpand:
            self.data = self.expand(withId=withId)
        return self.data
  

In [120]:
from pymongo import MongoClient
import pymongo
from django.http.response import JsonResponse
from bson.json_util import dumps, loads

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['CeDRI_UGV']['BatteryApplication_battery'].aggregate([
    {
        '$sort': {
            'dateTime': -1
        }
    }, {
        '$limit': 1
    }
])

list(result)[0]

{'_id': ObjectId('6390204933848362df900762'),
 'dateTime': datetime.datetime(2022, 12, 7, 5, 10, 33, 186000),
 'Sensor': {'_id': None, 'Voltage': 1.0, 'Current': None, 'Temperature': None},
 'Calculate': None}

In [ ]:
battery = readURL("http://127.0.0.1:8000/battery/battery")
df = battery.dataFrame(withId=False)
df

In [ ]:

import requests
import pandas as pd
import json
import ast

class readJSON:
    def __init__(self, json, pandas: pd.DataFrame):
        self.json = json
        if pandas == None:
            self.pandas = pd.read_json(self.json)

    def only_dict(self, d):
        return ast.literal_eval(d)

    def list_of_dicts(self, ld):
        return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

    def expand(self, withId:bool = True) -> pd.DataFrame:
        data = self.pandas
        for x in data.columns:
            if data[x].dtypes == object and data[x].name != '_id' :
                if withId:
                    newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).add_prefix(str(data[x].name) + '.')
                else:
                    newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).drop(columns=['_id']).add_prefix(str(data[x].name) + '.')
                data = data.drop(columns=data[x].name)
                data = data.join([newColumn])
        return data

    def dataFrame(self, withId:bool = True, columnExpand:bool = True) -> pd.DataFrame:
        data = self.pandas
        if not withId:
            self.pandas = self.pandas.drop(columns=['_id'])
        if columnExpand:
            self.pandas = self.expand(withId=withId)
        self.json = data.to_json()
        return data

In [ ]:
json_ = readJSON(battery.json())
json_.dataFrame(withId=False)